In [59]:
import pandas as pd
import json

In [38]:
data = pd.read_json('./data/tables.json')

In [42]:
data.head()

,column_names,column_names_original,column_types,db_id,foreign_keys,primary_keys,table_names,table_names_original
0,"[[-1, *], [0, perpetrator id], [0, people id],...","[[-1, *], [0, Perpetrator_ID], [0, People_ID],...","[text, number, number, text, number, text, tex...",perpetrator,"[[2, 9]]","[1, 9]","[perpetrator, people]","[perpetrator, people]"
1,"[[-1, *], [0, building], [0, room number], [0,...","[[-1, *], [0, building], [0, room_number], [0,...","[text, text, text, number, text, text, number,...",college_2,"[[9, 4], [13, 4], [19, 1], [20, 2], [15, 7], [...","[1, 4, 7, 11, 15, 22, 27, 31, 37, 39, 45]","[classroom, department, course, instructor, se...","[classroom, department, course, instructor, se..."
2,"[[-1, *], [0, id], [0, city], [0, country], [0...","[[-1, *], [0, id], [0, City], [0, Country], [0...","[text, number, text, text, text, text, text, n...",flight_company,"[[20, 7], [19, 1]]","[1, 7, 13]","[airport, operate company, flight]","[airport, operate_company, flight]"
3,"[[-1, *], [0, institution id], [0, name], [0, ...","[[-1, *], [0, instID], [0, name], [0, country]...","[text, number, text, text, number, text, text,...",icfp_1,"[[11, 7], [10, 1], [9, 4]]","[1, 4, 7, 9]","[institution, authors, papers, authorship count]","[Inst, Authors, Papers, Authorship]"
4,"[[-1, *], [0, body builder id], [0, people id]...","[[-1, *], [0, Body_Builder_ID], [0, People_ID]...","[text, number, number, number, number, number,...",body_builder,"[[2, 6]]","[1, 6]","[body builder, people]","[body_builder, people]"


In [ ]:
"""
datatabase = {
    "db_id": , data.db_id
    "tables": tables
    "foreign_keys": fks
    }

fks = [fk]

fk = {
    "child_table": ,
    "child_column": ,
    "parent_table": ,
    "parent_column": 
    }

tables = 
    { "table_name":
        [column]
    }
    

column = {
    "col_name": ,
    "column_types": 
}
"""

In [25]:
fks = []
for foreign_key in data.foreign_keys:
    fk = {
        "child_table": data.table_names_original[data.column_names_original[foreign_key[0]][0]],
        "child_column": data.column_names_original[foreign_key[0]][1],
        "parent_table": data.table_names_original[data.column_names_original[foreign_key[1]][0]],
        "parent_column": data.column_names_original[foreign_key[1]][1]
    }
    fks.append(fk)

In [26]:
fks

[{'child_table': 'perpetrator',
  'child_column': 'People_ID',
  'parent_table': 'people',
  'parent_column': 'People_ID'}]

In [28]:
data.column_types[0]

'text'

In [34]:
tables = {}
for tab in data.table_names_original:
    tables[tab] = []

for col_idx in range(1,len(data.column_types)):
    col = data.column_names_original[col_idx]
    column = {
        "col_name": col[1],
        "column_types": data.column_types[col_idx]
      }
    tables[data.table_names_original[col[0]]].append(column)

In [ ]:
dbs[data.db_id] = {}
database = {
    "db_id": data.db_id,
    "tables": tables,
    "foreign_keys": fks
}
dbs.append(database)

In [57]:
dbs = {}
for db_idx in range(len(data)):
    db = data.loc[db_idx]
    # print(db)
    # break
    fks = []
    for foreign_key in db.foreign_keys:
        fk = {
            "child_table": db.table_names_original[db.column_names_original[foreign_key[0]][0]],
            "child_column": db.column_names_original[foreign_key[0]][1],
            "parent_table": db.table_names_original[db.column_names_original[foreign_key[1]][0]],
            "parent_column": db.column_names_original[foreign_key[1]][1]
        }
    fks.append(fk)
    
    tables = {}
    for tab in db.table_names_original:
        tables[tab] = []

    for col_idx in range(1,len(db.column_types)):
        col = db.column_names_original[col_idx]
        column = {
            "col_name": col[1],
            "column_types": db.column_types[col_idx]
        }
        tables[db.table_names_original[col[0]]].append(column)
    database = {
        "tables": tables,
        "foreign_keys": fks
    }
    dbs[db.db_id] = database

In [63]:
res = json.dumps(dbs, indent=4)

In [64]:
with open('./data/spider.json','w') as f:
    f.write(res)
    f.close()